In [1]:
# Importar librerías necesarias para web scraping de Google Play Store
import requests
from bs4 import BeautifulSoup
import pandas as pd
import json
import time
import re
from urllib.parse import urljoin, urlparse
import random
from datetime import datetime

print("Librerías importadas correctamente!")
print("Fecha de ejecución:", datetime.now().strftime("%Y-%m-%d %H:%M:%S"))

Librerías importadas correctamente!
Fecha de ejecución: 2025-08-19 17:17:57


In [2]:
# Método alternativo más efectivo usando google-play-scraper
from google_play_scraper import app, reviews, Sort
import pandas as pd

class PlayStoreScraperAdvanced:
    def __init__(self):
        self.app_data = None
        self.reviews_data = []
    
    def get_app_info(self, app_id):
        """Obtiene información general de la aplicación"""
        try:
            self.app_data = app(app_id, lang='es', country='mx')
            print(f"Aplicación: {self.app_data['title']}")
            print(f"Desarrollador: {self.app_data['developer']}")
            print(f"Calificación promedio: {self.app_data['score']}")
            print(f"Total de reseñas: {self.app_data['reviews']}")
            return self.app_data
        except Exception as e:
            print(f"Error al obtener información de la app: {e}")
            return None
    
    def scrape_reviews(self, app_id, count=100, sort_by=Sort.NEWEST):
        """
        Extrae comentarios de una aplicación
        
        Parámetros:
        - app_id: ID de la aplicación (ej: 'com.whatsapp')
        - count: Número de comentarios a extraer
        - sort_by: Criterio de ordenamiento (NEWEST, HELPFUL, RATING)
        """
        try:
            print(f"Extrayendo {count} comentarios de {app_id}...")
            
            # Obtener comentarios
            result, continuation_token = reviews(
                app_id,
                lang='es',
                country='mx',
                sort=sort_by,
                count=count
            )
            
            # Procesar los comentarios
            processed_reviews = []
            for review in result:
                processed_review = {
                    'usuario': review['userName'],
                    'calificacion': review['score'],
                    'texto': review['content'],
                    'fecha': review['at'].strftime('%Y-%m-%d %H:%M:%S'),
                    'likes': review['thumbsUpCount'],
                    'version_app': review['appVersion'],
                    'id_comentario': review['reviewId']
                }
                processed_reviews.append(processed_review)
            
            self.reviews_data = processed_reviews
            print(f"¡Éxito! Se extrajeron {len(processed_reviews)} comentarios")
            return processed_reviews
            
        except Exception as e:
            print(f"Error al extraer comentarios: {e}")
            return []
    
    def save_to_csv(self, filename=None):
        """Guarda los comentarios en un archivo CSV"""
        if not self.reviews_data:
            print("No hay comentarios para guardar")
            return
        
        if not filename:
            timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
            filename = f"comentarios_playstore_{timestamp}.csv"
        
        df = pd.DataFrame(self.reviews_data)
        df.to_csv(filename, index=False, encoding='utf-8-sig')
        print(f"Comentarios guardados en: {filename}")
        
        # Mostrar estadísticas básicas
        print("\n📊 Estadísticas de los comentarios:")
        print(f"Total de comentarios: {len(df)}")
        print(f"Calificación promedio: {df['calificacion'].mean():.2f}")
        print("\nDistribución de calificaciones:")
        print(df['calificacion'].value_counts().sort_index())
        
        return df
    
    def get_sample_reviews(self, n=5):
        """Muestra una muestra de los comentarios extraídos"""
        if not self.reviews_data:
            print("No hay comentarios para mostrar")
            return
        
        df = pd.DataFrame(self.reviews_data)
        print(f"\n📝 Muestra de {min(n, len(df))} comentarios:")
        print("=" * 80)
        
        for i, row in df.head(n).iterrows():
            print(f"\n👤 Usuario: {row['usuario']}")
            print(f"⭐ Calificación: {row['calificacion']}/5")
            print(f"📅 Fecha: {row['fecha']}")
            print(f"💬 Comentario: {row['texto'][:200]}{'...' if len(row['texto']) > 200 else ''}")
            print("-" * 80)

# Crear instancia del scraper avanzado
advanced_scraper = PlayStoreScraperAdvanced()
print("Scraper avanzado de Google Play Store listo! 🚀")

Scraper avanzado de Google Play Store listo! 🚀


In [4]:
# 🎯 EXTRAER 5000 COMENTARIOS NEGATIVOS (≤ 3 ESTRELLAS)

# CONFIGURACIÓN - Solo comentarios con quejas (≤ 3 estrellas)
app_id = "com.bcp.innovacxion.yapeapp"  # ID de la app que ya usaste
num_comentarios_negativos = 9000  # OBJETIVO: 5000 comentarios negativos
lote_size = 1000  # Extraer lotes más grandes para filtrar más comentarios
filtro_estrellas = 3  # Solo comentarios con ≤ 3 estrellas

print(f"🎯 Extrayendo {num_comentarios_negativos} COMENTARIOS NEGATIVOS (≤ {filtro_estrellas} estrellas) de {app_id}")
print(f"🔍 Esto aumentará la proporción de quejas y comentarios críticos")

# Crear nuevo scraper si es necesario
try:
    advanced_scraper
except NameError:
    from google_play_scraper import app, reviews, Sort
    advanced_scraper = PlayStoreScraperAdvanced()

# EXTRACCIÓN CON FILTRO DE COMENTARIOS NEGATIVOS
import time
comentarios_negativos_filtrados = []
total_comentarios_revisados = 0
lote_actual = 1

# Usar múltiples criterios para capturar más variedad de comentarios negativos
criterios_busqueda = [Sort.RATING, Sort.NEWEST, Sort.MOST_RELEVANT]
nombres_criterios = ["Por Calificación (peores primero)", "Más Recientes", "Más Relevantes"]

print("🚀 Iniciando extracción con filtrado de comentarios negativos...")

for criterio, nombre_criterio in zip(criterios_busqueda, nombres_criterios):
    if len(comentarios_negativos_filtrados) >= num_comentarios_negativos:
        break
    
    print(f"\n🔄 Usando criterio: {nombre_criterio}")
    
    # Extraer más comentarios para poder filtrar
    comentarios_a_revisar = min(2000, (num_comentarios_negativos - len(comentarios_negativos_filtrados)) * 3)
    
    try:
        # Extraer comentarios del criterio actual
        result, continuation_token = reviews(
            app_id,
            lang='es',
            country='mx',
            sort=criterio,
            count=comentarios_a_revisar
        )
        
        print(f"📦 Revisando {len(result)} comentarios para filtrar negativos...")
        
        # FILTRAR SOLO COMENTARIOS NEGATIVOS (≤ 3 estrellas)
        comentarios_negativos_lote = []
        for review in result:
            total_comentarios_revisados += 1
            
            # FILTRO PRINCIPAL: Solo comentarios con ≤ 3 estrellas
            if review['score'] <= filtro_estrellas:
                # Usar el ID original de Google Play para evitar duplicados
                review_id_original = review['reviewId']
                
                # Evitar duplicados verificando el ID original
                ids_originales_existentes = [c.get('id_original') for c in comentarios_negativos_filtrados]
                if review_id_original not in ids_originales_existentes:
                    comentario_negativo = {
                        'id_original': review_id_original,  # Para control de duplicados
                        'comentario': review['content']
                    }
                    comentarios_negativos_lote.append(comentario_negativo)
        
        # Agregar comentarios negativos encontrados
        comentarios_negativos_filtrados.extend(comentarios_negativos_lote)
        
        print(f"✅ Comentarios negativos encontrados: {len(comentarios_negativos_lote)}")
        print(f"📊 Total negativos acumulados: {len(comentarios_negativos_filtrados)}/{num_comentarios_negativos}")
        print(f"🔍 Total comentarios revisados: {total_comentarios_revisados}")
        
        # Pausa entre criterios para evitar bloqueos
        if len(comentarios_negativos_filtrados) < num_comentarios_negativos:
            print("⏸️ Pausa de 3 segundos para evitar bloqueos...")
            time.sleep(3)
            
    except Exception as e:
        print(f"❌ Error con criterio '{nombre_criterio}': {e}")
        print("💡 Continuando con el siguiente criterio...")
        continue

# Limitar al número objetivo si tenemos más comentarios negativos
if len(comentarios_negativos_filtrados) > num_comentarios_negativos:
    comentarios_negativos_filtrados = comentarios_negativos_filtrados[:num_comentarios_negativos]

print(f"\n🎉 EXTRACCIÓN DE COMENTARIOS NEGATIVOS COMPLETADA!")
print(f"📊 Total de comentarios negativos extraídos: {len(comentarios_negativos_filtrados)}")
print(f"🔍 Total de comentarios revisados para filtrar: {total_comentarios_revisados}")

# Calcular eficiencia del filtro
if total_comentarios_revisados > 0:
    porcentaje_negativos = (len(comentarios_negativos_filtrados) / total_comentarios_revisados) * 100
    print(f"📈 Porcentaje de comentarios negativos encontrados: {porcentaje_negativos:.1f}%")

# Crear DataFrame solo con comentarios negativos: ID + Comentario
if comentarios_negativos_filtrados:
    df_negativos = pd.DataFrame(comentarios_negativos_filtrados)
    
    # CREAR ID INCREMENTAL (1, 2, 3, 4...)
    df_negativos['id'] = range(1, len(df_negativos) + 1)
    
    # Reorganizar columnas: ID incremental + comentario (sin el id_original)
    df_final_negativos = df_negativos[['id', 'comentario']].copy()
    
    print(f"✅ ¡Éxito! Dataset de comentarios negativos creado con {len(df_final_negativos)} comentarios")
    print(f"🔢 IDs asignados: 1 hasta {len(df_final_negativos)}")
    
    # Guardar archivo de comentarios negativos con timestamp
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    archivo_negativos = f"comentarios_negativos_{len(df_final_negativos)}_{timestamp}.csv"
    df_final_negativos.to_csv(archivo_negativos, index=False, encoding='utf-8-sig')
    
    print(f"💾 Comentarios negativos guardados en: {archivo_negativos}")
    
    # Mostrar estadísticas de comentarios negativos
    print(f"\n📊 ESTADÍSTICAS DE COMENTARIOS NEGATIVOS:")
    print(f"📝 Columnas extraídas: {list(df_final_negativos.columns)}")
    print(f"🎯 Total de quejas/comentarios críticos: {len(df_final_negativos)}")
    print(f"🔢 IDs: Numeración del 1 al {len(df_final_negativos)}")
    print(f"📄 Comentario más largo: {df_final_negativos['comentario'].str.len().max()} caracteres")
    print(f"📄 Comentario más corto: {df_final_negativos['comentario'].str.len().min()} caracteres")
    print(f"📄 Longitud promedio: {df_final_negativos['comentario'].str.len().mean():.0f} caracteres")
    
    # Mostrar muestra de comentarios negativos
    print(f"\n📝 MUESTRA DE 5 COMENTARIOS NEGATIVOS:")
    print("=" * 80)
    for i, row in df_final_negativos.head(5).iterrows():
        comentario_preview = row['comentario'][:200] + "..." if len(row['comentario']) > 200 else row['comentario']
        print(f"\n🔢 ID: {row['id']}")
        print(f"💬 Comentario: {comentario_preview}")
        print("-" * 50)
    
    # Mostrar el DataFrame de comentarios negativos
    print(f"\n📊 PRIMERAS 10 FILAS DEL DATASET DE COMENTARIOS NEGATIVOS:")
    display(df_final_negativos.head(10))
    
    # Verificar duplicados (ya no debería haber porque usamos IDs únicos incrementales)
    print(f"\n✅ IDs incrementales únicos generados correctamente")
    print(f"🔢 Rango de IDs: 1 - {len(df_final_negativos)}")
    
    # Resumen final optimizado para quejas
    print(f"\n🎯 RESUMEN FINAL - COMENTARIOS NEGATIVOS:")
    print(f"   ✅ Comentarios negativos obtenidos: {len(df_final_negativos)}")
    print(f"   🔢 IDs: Numeración incremental del 1 al {len(df_final_negativos)}")
    print(f"   📊 Filtro aplicado: ≤ {filtro_estrellas} estrellas")
    print(f"   📈 Eficiencia del filtro: {porcentaje_negativos:.1f}% de comentarios fueron negativos")
    print(f"   💾 Archivo principal: {archivo_negativos}")
    
else:
    print("❌ No se pudieron extraer comentarios negativos")
    print(f"💡 La app '{app_id}' podría tener muy pocos comentarios con ≤ {filtro_estrellas} estrellas")
    print("💡 Soluciones:")
    print("   - Intenta con filtro_estrellas = 4 (incluir comentarios neutros)")
    print("   - Prueba con otra aplicación que tenga más quejas")
    print("   - Verifica tu conexión a internet")

🎯 Extrayendo 9000 COMENTARIOS NEGATIVOS (≤ 3 estrellas) de com.bcp.innovacxion.yapeapp
🔍 Esto aumentará la proporción de quejas y comentarios críticos
🚀 Iniciando extracción con filtrado de comentarios negativos...

🔄 Usando criterio: Por Calificación (peores primero)
📦 Revisando 2000 comentarios para filtrar negativos...
✅ Comentarios negativos encontrados: 0
📊 Total negativos acumulados: 0/9000
🔍 Total comentarios revisados: 2000
⏸️ Pausa de 3 segundos para evitar bloqueos...

🔄 Usando criterio: Más Recientes
📦 Revisando 2000 comentarios para filtrar negativos...
✅ Comentarios negativos encontrados: 442
📊 Total negativos acumulados: 442/9000
🔍 Total comentarios revisados: 4000
⏸️ Pausa de 3 segundos para evitar bloqueos...

🔄 Usando criterio: Más Relevantes
📦 Revisando 2000 comentarios para filtrar negativos...
✅ Comentarios negativos encontrados: 1022
📊 Total negativos acumulados: 1464/9000
🔍 Total comentarios revisados: 6000
⏸️ Pausa de 3 segundos para evitar bloqueos...

🎉 EXTRACC

,id,comentario
0,1,no puedo meter el numero de celular para habri...
1,2,no suena cuando llega un yape
2,3,me gustaría saber x k no suena cuando me yapea
3,4,no puedo entrar pork no puedo
4,5,Lo único malo de Yape es que si por error yape...
5,6,no quiere entrar yape
6,7,no puedo usarlo
7,8,Por qué no me permite activar la biometría dig...
8,9,mala está ap
9,10,no puedo crear la cuenta



✅ IDs incrementales únicos generados correctamente
🔢 Rango de IDs: 1 - 1464

🎯 RESUMEN FINAL - COMENTARIOS NEGATIVOS:
   ✅ Comentarios negativos obtenidos: 1464
   🔢 IDs: Numeración incremental del 1 al 1464
   📊 Filtro aplicado: ≤ 3 estrellas
   📈 Eficiencia del filtro: 24.4% de comentarios fueron negativos
   💾 Archivo principal: comentarios_negativos_1464_20250819_172043.csv


# 🎯 FILTRO DE COMENTARIOS NEGATIVOS - EXPLICACIÓN

## 🔍 **¿Qué hace este código modificado?**

### **1. Filtro Principal**
- ✅ **Solo extrae comentarios con ≤ 3 estrellas**
- ❌ **Descarta comentarios con 4 y 5 estrellas** (positivos)
- 🎯 **Resultado**: Mayor proporción de quejas y comentarios críticos

### **2. Estrategia Multi-Criterio**
```python
criterios_busqueda = [Sort.RATING, Sort.NEWEST, Sort.MOST_RELEVANT]
```
- **Por Calificación**: Los peores comentarios primero
- **Más Recientes**: Quejas actuales y problemas nuevos
- **Más Relevantes**: Comentarios críticos importantes

### **3. Eficiencia del Proceso**
- 📦 **Extrae lotes grandes** (1000-2000 comentarios)
- 🔍 **Filtra solo los negativos** (≤ 3 estrellas)
- 💾 **Guarda solo los que necesitas**
- 📊 **Muestra estadísticas de eficiencia**

## ⚙️ **Configuración Ajustable**

Si quieres cambiar el filtro, modifica esta línea:
```python
filtro_estrellas = 3  # Cambia a 4 para incluir comentarios neutros
```

**Opciones:**
- `filtro_estrellas = 1` → Solo comentarios muy negativos (1 estrella)
- `filtro_estrellas = 2` → Comentarios muy negativos (1-2 estrellas)  
- `filtro_estrellas = 3` → Comentarios negativos (1-3 estrellas) **← ACTUAL**
- `filtro_estrellas = 4` → Incluir comentarios neutros (1-4 estrellas)

## 📊 **Archivos Generados**
- `comentarios_negativos_[número]_[timestamp].csv` - Solo quejas y críticas
- `comentarios_negativos_limpio_[número]_[timestamp].csv` - Sin duplicados

## 🎉 **Ventajas de este Filtro**
- 🎯 **Mayor relevancia**: Solo comentarios con problemas
- 📈 **Mejor proporción**: Más quejas, menos comentarios positivos
- 💡 **Insights valiosos**: Identifica problemas reales de la app
- 🔍 **Análisis enfocado**: Datos más útiles para mejoras